# Convert old input card

1. meta and experiment

In [5]:
from ruamel.yaml import YAML
from cvm.utils import get_inp
import sys

yaml = YAML()
yaml.indent(mapping=4, sequence=4, offset=2)
yaml.default_flow_style = None
yaml.width = 120

In [21]:
inp = get_inp('<old_input_card.json>')

In [22]:
meta = dict(host=inp['host'], impurity=inp['impurity'], prefix=inp['prefix'], description=inp['description'], structure=inp['structure'])
experiment = dict(temperature=inp['experiment'][0]['temp'], concentration=inp['experiment'][0]['c'])

tmp = {'meta': meta, 'experiment': experiment}

In [23]:
tmp

{'meta': {'host': 'Pd',
  'impurity': 'Ru',
  'prefix': '',
  'description': 'PdRu solubility limit calculation with tetrahedron approximation',
  'structure': 'A1'},
 'experiment': {'temperature': [1800, 1600, 1400, 1200, 1000],
  'concentration': [0.1615, 0.12, 0.078, 0.045, 0.029]}}

In [24]:
with open('input.yml', 'w') as f:
    yaml.dump(tmp, f)

2. enegires

In [25]:
def extractor(s, prefix):
    print(s['label'])
    print(s['transfer'])
    print(s['temp'])
    data = s['datas']
    lattice = data['lattice_c']
    host=data['host_en']
    
    n_ens = {}
    for i in range(11):
        s_i = str(i + 1)
        l = 'pair' + s_i
        n_ens[s_i + '_II'] = data[l][0]['energy']
        n_ens[s_i + '_IH'] = data[l][1]['energy']
        n_ens[s_i + '_HH'] = data[l][2]['energy']
    
    normalizer = dict(lattice=lattice, **n_ens)
    clusters = dict(
        lattice=lattice,
        host=host,
        Rh4=data['tetra'][0]['energy'],
        Rh3Pd1=data['tetra'][1]['energy'],
        Rh2Pd2=data['tetra'][2]['energy'],
        Rh1Pd3=data['tetra'][3]['energy'],
        Pd4=data['tetra'][4]['energy'],
    )
    
    n_name = prefix + '_normalizer.csv'
    c_name = prefix + '_clusters.csv'
    
    print(n_name)
    print(c_name)
    print()
    
    pd.DataFrame(normalizer).to_csv(n_name, index=False)
    pd.DataFrame(clusters).to_csv(c_name, index=False)
    

In [26]:
for i, s in enumerate(inp['series']):
    extractor(s, str(i))

$T_\mathrm{FD}=800$K
[[1, 11, 2]]
[400, 1290, 50]
0_normalizer.csv
0_clusters.csv

$T_\mathrm{FD}=1000$K
[[1, 11, 2]]
[400, 1550, 50]
1_normalizer.csv
1_clusters.csv

$T_\mathrm{FD}=1200$K
[[1, 11, 2]]
[400, 1700, 50]
2_normalizer.csv
2_clusters.csv

$T_\mathrm{FD}=1400$K
[[1, 11, 2]]
[500, 1700, 50]
3_normalizer.csv
3_clusters.csv

$T_\mathrm{FD}=1600$K
[[1, 11, 2]]
[500, 1870, 50]
4_normalizer.csv
4_clusters.csv

